# Mountain glaciers

Here we'll show how to simulate a mountain glacier.
This will be more complicated than the previous demo because we need to track an advancing and retreating terminus.

In [ ]:
from tqdm.notebook import trange, tqdm
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import firedrake
from firedrake import inner, grad, dx, exp, min_value, max_value, Constant
import irksome
from icepack2 import model

## Geometry

I got some numbers for Mt. Rainier in WA from [this volume](http://npshistory.com/publications/mora/glaciers-1963.pdf) from 1963, edited by Mark Meier and featuring many contributions by Austin Post.

In [ ]:
radius = Constant(12e3)
mesh = firedrake.UnitDiskMesh(4)
mesh.coordinates.dat.data[:] *= float(radius)

Mostly the same function spaces as before.

In [ ]:
dg0 = firedrake.FiniteElement("DG", "triangle", 0)
cg1 = firedrake.FiniteElement("CG", "triangle", 1)
S = firedrake.FunctionSpace(mesh, cg1)
Q = firedrake.FunctionSpace(mesh, dg0)
V = firedrake.VectorFunctionSpace(mesh, cg1)
Σ = firedrake.TensorFunctionSpace(mesh, dg0, symmetry=True)

Mt. Rainier is about 4km high.
We'll use a bed topography of the form $B\exp(-|x|^2/r^2)$, although this is actually a little unrealistic.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)

B = Constant(4e3)
r_b = Constant(150e3 / (2 * π))
expr = ...
b = firedrake.Function(S).interpolate(expr)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_title("Bed elevation")
colors = firedrake.tripcolor(b, axes=ax)
fig.colorbar(colors);

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
firedrake.trisurf(b, axes=ax);

For mountain glaciers, we have to think very hard about what the right surface mass balance field is.
The measurements from the 60s found that:
* at 1600m elevation, the SMB is about -8.7 m/yr of water equivalent
* the accumulation maxes out at 0.7 m/yr at the peak elevation

With these values, we can make up a reasonable SMB field that is linear with elevation up to some maximum.

In [ ]:
z_measured = Constant(1600.0)
a_measured = Constant(-0.917 * 8.7)
a_top = Constant(0.7)
z_top = Constant(4e3)
δa_δz = (a_top - a_measured) / (z_top - z_measured)
a_max = Constant(0.7)

def smb(z):
    return min_value(a_max, a_measured + δa_δz * (z - z_measured))

We'll start with a very artificial initial guess for the thickness and evolve it toward equilibrium.

In [ ]:
r_h = Constant(5e3)
H = Constant(100.0)
expr = H * firedrake.max_value(0, 1 - inner(x, x) / r_h**2)
h = firedrake.Function(Q).interpolate(expr)
h_0 = h.copy(deepcopy=True)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(h, cmap="Blues", axes=ax)
fig.colorbar(colors);

We can then compute the initial surface elevation and mass balance.

In [ ]:
a = firedrake.Function(Q).interpolate(smb(b + h))

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(a, vmin=-1.0, vmax=1.0, cmap="RdBu", axes=ax)
fig.colorbar(colors);

## Initial velocity computation

In order to calculate the material properties of the glacier, we'll assume that it's roughly temperate, which isn't too far off from the truth.

In [ ]:
from icepack2.constants import gravity, ice_density, glen_flow_law
A = Constant(158.8)
n = Constant(glen_flow_law)
τ_c = Constant(0.1)
ε_c = ...

Unlike the ice shelf case, there is no real boundary, so we don't have to come up with some initial velocity.

In [ ]:
Z = V * Σ * V
z = firedrake.Function(Z)

For a grounded glacier, we hvae to also add the basal shear stress as an unknown and make some decision about the friction coefficient.
To make the SSA model effectively do SIA, what we need is that, in the sliding law
$$K|\tau|^{m - 1}\tau = -u,$$
we take
$$K = \frac{hA}{n + 2}.$$
We also define the sliding coefficient as
$$K = u_c / \tau_c^m$$
where $\tau_c$ is a reference stress and $u_c$ a reference speed.
We're using the same reference stress for the flow and sliding laws of $\tau_c = $ 100 kPa, so we can then compute the reference speed as $u_c = K\tau_c^m$.
We can then add more sliding on top of the effective deformation that we use to mimic SIA.
To add some extra real sliding, we've increased the critical speed $u_c$ by a factor of 100 m/yr.

In [ ]:
K = h * A / (n + 2)
U_c = Constant(100.0)
u_c = ...

For the ice shelf case, we used a flow law with $n = 3$.
Here we're going to use a combination of rheologies with both $n = 1$ and $n = 3$.
It helps to wrap all this up into some dictionaries to make things easier.

In [ ]:
glen_rheology = {
    "flow_law_exponent": n,
    "flow_law_coefficient": ε_c / τ_c ** n,
    "sliding_exponent": n,
    "sliding_coefficient": u_c / τ_c ** n,
}

α = firedrake.Constant(1e-4)
linear_rheology = {
    "flow_law_exponent": 1,
    "flow_law_coefficient": ε_c / τ_c,
    "sliding_exponent": 1,
    "sliding_coefficient": u_c / τ_c,
}

Now we'll create the parameters that we need for the solver, including what quadrature degree and solution strategy to use.

In [ ]:
degree = 1
qdegree = max(8, degree ** glen_flow_law)
pparams = {"form_compiler_parameters": {"quadrature_degree": qdegree}}

sparams = {
    "solver_parameters": {
        "snes_monitor": None,
        "snes_type": "newtonls",
        "snes_max_it": 200,
        "snes_linesearch_type": "nleqerr",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
}

The momentum balance form has more terms.
But the process is essentially the same as before.

In [ ]:
from icepack2 import model
from icepack2.model.variational import momentum_balance, flow_law, friction_law

u, M, τ = firedrake.split(z)
v, N, σ = firedrake.TestFunctions(Z)

F = ...

momentum_problem = firedrake.NonlinearVariationalProblem(F, z, **pparams)
momentum_solver = firedrake.NonlinearVariationalSolver(momentum_problem, **sparams)

Now we can see our initial value of the velocity.

In [ ]:
momentum_solver.solve()

In [ ]:
u_init, M_init, τ_init = z.subfunctions

fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(u_init, axes=axes)
fig.colorbar(colors);

## Forward modeling

Now make an even bigger function space, God help us all.
Pack in the initial values of the velocity, membrane stress, and basal stress that we computed before.

In [ ]:
W = ...
w = firedrake.Function(W)
w.sub(0).assign(u_init)
w.sub(1).assign(M_init)
w.sub(2).assign(τ_init)
w.sub(3).assign(h);

Again, we form the momentum and mass balance equations, add them together into one big form, and pass it to Irksome.

In [ ]:
u, M, τ, h = ...
v, N, σ, ϕ = ...

K = h * A / (n + 2)
U_c = Constant(100.0)
u_c = K * τ_c ** n + U_c

glen_rheology = {
    "flow_law_exponent": n,
    "flow_law_coefficient": ε_c / τ_c ** n,
    "sliding_exponent": n,
    "sliding_coefficient": u_c / τ_c ** n,
}

α = firedrake.Constant(1e-4)
linear_rheology = {
    "flow_law_exponent": 1,
    "flow_law_coefficient": ε_c / τ_c,
    "sliding_exponent": 1,
    "sliding_coefficient": u_c / τ_c,
}

F_momentum = ...
F_mass = ...

F = F_momentum + F_mass

In [ ]:
tableau = irksome.BackwardEuler()
t = Constant(0.0)
dt = Constant(1.0)

lower = firedrake.Function(W)
upper = firedrake.Function(W)
lower.assign(-np.inf)
upper.assign(+np.inf)
lower.subfunctions[3].assign(0.0)
bounds = ("stage", lower, upper)

bparams = {
    "solver_parameters": {
        "snes_monitor": ":rainier-output-vi.log",
        "snes_type": "vinewtonrsls",
        "snes_max_it": 200,
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
    "form_compiler_parameters": {"quadrature_degree": 6},
    "stage_type": "value",
    "basis_type": "Bernstein",
    "bounds": bounds,
}

solver = irksome.TimeStepper(...)

The good part: run the model for 500 years of simulation time.

In [ ]:
hs = [w.subfunctions[3].copy(deepcopy=True)]

final_time = 500.0
num_steps = int(final_time / float(dt))
for step in trange(num_steps):
    solver.advance()
    h = w.subfunctions[3]
    a.interpolate(smb(b + h))
    hs.append(h.copy(deepcopy=True))

Plot the final velocity.
This speed is pretty slow -- Rainier is ~20x faster than this.

In [ ]:
u, M, τ, h = w.subfunctions

fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.set_xlim((0, 10e3))
axes.set_ylim((0, 10e3))
colors = firedrake.tripcolor(u, axes=axes)
fig.colorbar(colors);

Make a movie to show the glacier evolution.

In [ ]:
%%capture

fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.set_xlim((0, 10e3))
axes.set_ylim((0, 10e3))
colors = firedrake.tripcolor(
    hs[0], vmax=130.0, num_sample_points=1, shading="gouraud", cmap="Blues", axes=axes
)
fig.colorbar(colors);

In [ ]:
from matplotlib.animation import FuncAnimation

fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=1)
def animate(h):
    colors.set_array(fn_plotter(h))

animation = FuncAnimation(fig, animate, tqdm(hs), interval=1e3/60)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

Plots of the volume show the glacier roughly attaining steady state.

In [ ]:
times = np.linspace(0.0, final_time, num_steps + 1)
volumes = [firedrake.assemble(h * dx) / 1e9 for h in hs]
fig, ax = plt.subplots()
ax.set_xlabel("Time (years)")
ax.set_ylabel("Ice volume (km${}^3$)")
ax.plot(times, volumes);

Compute the sizes of the accumulation and ablation areas.

In [ ]:
from firedrake import conditional, And
ablation_mask = ...
accumulation_mask = ...
ablation_area = firedrake.assemble(ablation_mask * dx)
accumulation_area = firedrake.assemble(accumulation_mask * dx)
print(f"Accumulation area: {accumulation_area / 1e6:0.0f} km²")
print(f"Ablation area:     {ablation_area / 1e6:0.0f} km²")

Save the results to disk so that we can do something else with them.

In [ ]:
with firedrake.CheckpointFile("mountain-glacier.h5", "w") as chk:
    chk.save_mesh(mesh)
    chk.save_function(u, name="velocity")
    chk.save_function(M, name="membrane_stress")
    chk.save_function(τ, name="basal_stress")
    chk.save_function(h, name="thickness")